# 2 - Cleaning and filtering scrapped dataset

## 1. Load dataset

### a) Load the .env variables

In [22]:
from dotenv import load_dotenv

load_dotenv()

True

### b) Import .csv file
- Manually move your scrapped dataframe to sam_files/dataframes/scrapped
- ⚠️ change line 17 for the .csv filename

In [23]:
import os
from pathlib import Path
import pandas as pd


def get_csv_path(filename):
    # Capture the root directory and construct the path (assuming a consistent project structure)
    root_dir = Path.cwd().parent.parent.parent.parent.parent  # Adjust if needed based on your actual structure
    GITHUB_PROFILE_NAME = os.getenv('GITHUB_PROFILE_NAME')
    base_path = root_dir / f"{GITHUB_PROFILE_NAME}/stable-audio-tools-sam/sam_files/dataframes/scrapped"

    # Combine the base path with the filename
    csv_path = base_path / filename
    return csv_path

# Use the function to get the path
filename = "Bon _Export_20k_full_tags.csv"
csv_path = get_csv_path(filename)

# Load the DataFrame
df = pd.read_csv(csv_path)

## 2. Process dataset

### a) Load the script to preprocess the .csv
- No worries, it takes more than one minute to run it !

In [24]:
"""
This cell defines a function to process a DataFrame by updating specific columns,
removing unnecessary ones, and filtering rows. It includes:
- Updating 'Genre', 'Mood', 'Movement', and 'Theme' columns based on keywords.
- Dropping unwanted columns and rows with odd indices.
- Reordering columns and filtering out rows with duration patterns in the 'Title' column.
"""

import re

# Define the function to perform all operations
def process_dataframe(df):
    # Define the function to update Genre and Mood
    def update_genre_mood(row):
        row['Genre'] = row['Keyword1']
        row['Mood'] = row['Keyword2']
        return row

    # Apply the function to update Genre and Mood
    df = df.apply(update_genre_mood, axis=1)

    # Define the function to update Movement and Theme
    def update_movement_theme(df):
        for i in range(len(df) - 1):
            df.at[i, 'Movement'] = df.at[i + 1, 'Keyword1']
            df.at[i, 'Theme'] = df.at[i + 1, 'Keyword2']
        return df

    # Apply the function to update Movement and Theme
    df = update_movement_theme(df)

    # Drop unnecessary columns
    df = df.drop(columns=['Category', 'Keyword1', 'Category.1', 'Keyword2'])

    # Remove rows with odd indices
    df = df.iloc[::2].reset_index(drop=True)

    # Reorder columns to place 'Other Keyword' and 'Other Keywords.1' as the last columns
    new_order = ['Title', 'Title_URL', 'Artist Name', 'Artist Link', 'Duration', 'Genre', 'Mood', 'Movement', 'Theme', 'Other keywords', 'Other keywords.1']
    df = df[new_order]

    return df

# Function to identify rows with durations in their names
def has_duration(name):
    # Regular expression to match patterns like '38s', '43 second', '43 seconds'
    return bool(re.search(r'\d+s\b|\bseconds?\b', name))

# Apply the processing function
df_processed = process_dataframe(df)

# Filter out rows with durations in their names
df_processed = df_processed[~df_processed['Title'].apply(has_duration)]

## 3. Create new dataframe

### a) Script for creating a specific_genre.csv 

In [25]:
# Function to create a DataFrame with a specified genre and its variations
def create_genre_df(df, genre_variations, columns, num_rows=500):
    # Ensure columns do not contain NaN values
    df = df[~df[columns].isna().any(axis=1)]

    # Search in 'Genre' column first
    pattern = '|'.join(genre_variations)
    genre_rows = df[df['Genre'].str.contains(pattern, case=False, regex=True)]

    # If we have enough rows, return them
    if len(genre_rows) >= num_rows:
        return genre_rows.head(num_rows)

    # If not enough, search in other columns
    remaining_rows_needed = num_rows - len(genre_rows)
    other_columns_rows = df[columns].apply(lambda x: x.str.contains(pattern, case=False, regex=True)).any(axis=1)

    # Exclude already selected rows from other columns
    additional_rows = df[other_columns_rows & ~df.index.isin(genre_rows.index)]

    # Combine genre rows with additional rows
    final_rows = pd.concat([genre_rows, additional_rows]).head(num_rows)

    return final_rows

### b) Set values
- line 2 : genre
- line 3 : genre variations
- line 4 : {num_rows} : {number of audio (rows) by dataframe}

In [26]:
genre = "lofi"
genre_variations = ['lofi', 'chill lofi']
num_rows = 500

### c) create num-rows_genre.csv

In [27]:
# List of columns to search in
columns_to_search = ['Mood', 'Movement', 'Theme', 'Other keywords', 'Other keywords.1']

# Insert "Dowloaded?" column filled with '0' between "Title_URL" and "Artist_name"
df.insert(2, 'Dowloaded?', 0)  # 2 is the position where the new column will be inserted

# Create DataFrames for classical and lofi genres
### Put your values (num_rows = number of rows by dataframe)
genre_df = create_genre_df(df_processed, genre_variations, columns_to_search, num_rows=num_rows)

### c) Save your num-rows_genre.csv

In [28]:
from pathlib import Path
import os

root_dir = Path.cwd().parent.parent.parent.parent.parent  # Adjust if needed based on your actual structure
GITHUB_PROFILE_NAME = os.getenv('GITHUB_PROFILE_NAME')
base_path = root_dir / f"{GITHUB_PROFILE_NAME}/stable-audio-tools-sam/sam_files/dataframes/filtered_by_genre"

# Create the directory if it doesn't exist
base_path.mkdir(parents=True, exist_ok=True)

# Save the DataFrames to CSV files
genre_df.to_csv(base_path / f'{num_rows}_{genre}_tracks.csv', index=False)

print(f"DataFrame saved to {base_path}")

DataFrame saved to /home/arthurcornelio/code/arthurcornelio88/stable-audio-tools-sam/sam_files/dataframes/filtered_by_genre


## 4. Keep going to the next notebook !